<a href="https://colab.research.google.com/github/MunJinSeo/MyProject/blob/master/daniel021_nsmc_koelectra_base_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 KoELECTRA를_이용한_감정분석기_학습 (Pytorch + HuggingFace)
# Colab 에서 개발 및 실행
# Github : https://github.com/MunJinSeo/MyProject/
<br>

## References 1
- 김희규님의 "HuggingFace KoElectra로 NSMC 감성분석 Fine-tuning해보기"<br>
https://heegyukim.medium.com/huggingface-koelectra%EB%A1%9C-nsmc-%EA%B0%90%EC%84%B1%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0-1a23a0c704af

- 이지원님의 Github : nlp_emotion_classification <br>
https://github.com/jiwonny/nlp_emotion_classification

## 사용모델 KoELECTRA , ELECTRA
- 한국어 : 박장원님의 KoElectra-small 사용<br>
https://monologg.kr/2020/05/02/koelectra-part1/<br>
https://github.com/monologg/KoELECTRA
- 영어 : 구글 ELECTRA - small 사용<br>
https://huggingface.co/google/electra-small-discriminator<br>
https://github.com/google-research/electra

## Dataset
- 한국어 : 네이버 영화 리뷰 데이터셋<br>
https://github.com/e9t/nsmc
- 영어 : Freinds <br>
http://doraemon.iis.sinica.edu.tw/emotionlines/

## References 2
- https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP
- https://blog.naver.com/horajjan/221739630055
<br>@@<br>
- https://github.com/YongWookHa/kor-text-preprocess
- https://github.com/likejazz/korean-sentence-splitter
- https://github.com/lovit/soynlp
<br>@@<br>
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249


## 기타
반드시 GPU로 실행 - Colab무료환경에서 1epoch 당 약 15~20분 소요

# 필요 lib 설치

In [1]:
# lib 설치
!pip install transformers
!pip install torch

     |████████████████████████████████| 1.5MB 15.7MB/s 
     |████████████████████████████████| 890kB 57.2MB/s 
     |████████████████████████████████| 2.9MB 76.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=408d0c7e7715825e686b9ac348460e3f368a9870cd56f99eea28ebe793007bab
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!pip install kss
!pip install konlpy
!pip install sentencepiece
!pip install soynlp

  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251580 sha256=376335668ec2627fce01782f0d863325323e38dda01cc6609fffae97b75efeee
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
Successfully built kss
     |████████████████████████████████| 19.4MB 339kB/s 
     |████████████████████████████████| 460kB 42.3MB/s 
     |████████████████████████████████| 92kB 15.9MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 1.1MB 14.4MB/s 
     |████████████████████████████████| 419kB 12.2MB/s 


In [3]:
# (미사용) Colab TPU 사용을 위해 설치
#--!pip install torch_xla
#--#@param ["1.5" , "20200325", "nightly"]
#VERSION = "1.7"
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version $VERSION

# NSMC 데이터셋 다운로드

In [4]:
#NSMC 데이터셋 다운로드

#!git clone https://github.com/e9t/nsmc.git
#학습데이터를 최대한 많이 하기 위해 전체 합쳐진 파일과 test파일을 사용함
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
#!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
# ko_data.csv #별도 kaggle에서 받아서 사용 https://www.kaggle.com/c/korean-sa-competition-bdc101

--2020-12-19 22:39:48--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  --.-KB/s    in 0.1s    

2020-12-19 22:39:48 (33.4 MB/s) - ‘ratings_test.txt’ saved [4893335/4893335]

--2020-12-19 22:39:48--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings

In [5]:
!head ratings.txt
#!head ratings_train.txt
!head ratings_test.txt
# !head ko_data.csv #별도 kaggle에서 받아서 사용 https://www.kaggle.com/c/korean-sa-competition-bdc101

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1
id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	0
7898805	음악이 주가 된, 최고의 음악영화	1
6315043	진정한 쓰레기	0
6097171	마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다	0
8932678	갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다	0


# 필요 모듈 import

In [6]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [7]:
# (미사용) TPU 사용을 위해 필요
#import torch_xla
#import torch_xla.core.xla_model as xm

In [8]:
import kss
import re
from soynlp.normalizer import *

In [9]:
# GPU or CPU
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print('No GPU available, using the CPU instead.')

# (미사용) TPU
#device = xm.xla_device()

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


# 데이터셋 처리 (Dataset Calss / 전처리)
(train data와 과제 sample data형식이 다르고, encoding이 다르기 때문에 분리 처리)

In [10]:
class NSMC_Dataset(Dataset):
  
  def __init__(self, csv_file, ftype):
    # train data와 sample data 각각 처리

    # 초기 전처리 1, 아래쪽 전처리 2 로 clean_text() 펑션 분리함 (학습 할때 line별 전처리 후 사용됨)
    # --- 전처리 1 start ------------------------

    if ftype == 'train':
      # 일부 값중에 NaN 제거
      self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0)

    elif ftype == 'sample':
      # 한글처리용 cp949적용
      self.dataset = pd.read_csv(csv_file, sep=',', encoding='cp949') 
       # 뒤쪽 컬럼 label 값이 없으므로 기본값으로 추가함
      self.dataset.insert(2,'Predicted','-')
      #print(self.dataset)
    else:
      self.dataset = pd.read_csv(csv_file, sep=',').dropna(axis=0)


    # for idx, document in self.dataset.iterrows():
    #   print(idx, document)
    self.sub1 = re.compile('[^ .?!/@$%~|0-9|ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기, 특수기호 일부를 제외한 모든 글자제거
    self.sub2 = re.compile('[\s]+')  # white space duplicate
    self.sub3 = re.compile('[\.]+')  # full stop duplicate

    #중복되는 문장 제거
    if ftype == 'sample':
      print('과제 제출용은 중복 문장 제거하면 안됨')
    else:
      # 중복제거: document가 리뷰 텍스트 내용의 title명이다
      self.dataset.drop_duplicates(subset=['document'], inplace=True)

    # dataset 확인
    print(self.dataset.describe())
    print(self.dataset)

    # tokenizer
    #self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")
    self.tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
    
    # --- 전처리 1 end ------------------------
  
  def __len__(self):
    return len(self.dataset)
  
  def clean_text(self, txt):
    # --- 전처리 2 start ----------------------
    cleaned = self.sub1.sub('', txt.strip())  # .strip()은 문장의 앞뒤 공백제거함
    cleaned = self.sub2.sub(' ', cleaned)
    cleaned = self.sub3.sub('.', cleaned)
    cleaned = emoticon_normalize(cleaned, num_repeats=3) # 감정 반복 단순화
    cleaned = repeat_normalize(cleaned, num_repeats=2) # 중복 글자 단순화
    #cleaned = only_text(cleaned) # text만 추출
    #cleaned = only_hangle(cleaned) # 한글만 추출
    #cleaned = only_hangle_number(cleaned) # 한글/숫자만 추출

    # 문장 분리하여 일정 길이 넘는 것만 사용
    ttStr = ""
    for ssStr in kss.split_sentences(cleaned):
      #print(ssStr)
      if len(ssStr) > 1:
        #ttStr += ssStr + " \n"
        ttStr += "[CLS] " + ssStr + " [SEP]"

    cleaned = ttStr
    # --- 전처리 2 end ------------------------
    return cleaned

  
  def __getitem__(self, idx):
    # 행번호별 컬럼 지정하여 할당
    row = self.dataset.iloc[idx, 1:3].values  # idx번째 행의 첫번째 컬럼 0을 제외하고 1~3컬럼 할당
    text = self.clean_text( txt=row[0] ) # 전처리 2 : clean_text()
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [11]:
train_dataset = NSMC_Dataset("ratings.txt","train")
test_dataset = NSMC_Dataset("ratings_test.txt","train")
sample_dataset = NSMC_Dataset("ko_data.csv","sample")

                 id          label
count  1.461820e+05  146182.000000
mean   6.779186e+06       0.498283
std    2.919223e+06       0.499999
min    3.300000e+01       0.000000
25%    4.814832e+06       0.000000
50%    7.581160e+06       0.000000
75%    9.274760e+06       1.000000
max    1.027815e+07       1.000000
              id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149995   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149996   8549745                                      평점이 너무 낮아서...      1
149997   9


                 id         label
count  4.915700e+04  49157.000000
mean   6.752945e+06      0.502695
std    2.937158e+06      0.499998
min    6.010000e+02      0.000000
25%    4.777143e+06      0.000000
50%    7.565415e+06      1.000000
75%    9.260204e+06      1.000000
max    1.027809e+07      1.000000
            id                                           document  label
0      6270596                                                굳 ㅋ      1
1      9274899                               GDNTOPCLASSINTHECLUB      0
2      8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3      6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4      6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
...        ...                                                ...    ...
49995  4608761          오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함      1
49996  5308387       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO      0
49997  9072549                 그림도 좋

In [12]:
tmpstr = '훌륭하다. 초한지 얼른 읽어보고 다시 봐야겠다. 연출 훌륭하다 껄껄 한신의 토사구팽은 슬펐다'
print( train_dataset.clean_text( txt = tmpstr) )

[CLS] 훌륭하다. [SEP][CLS] 초한지 얼른 읽어보고 다시 봐야겠다. [SEP][CLS] 연출 훌륭하다 [SEP][CLS] 껄껄 한신의 토사구팽은 슬펐다 [SEP]


# 모델 생성 (Create Model)

In [13]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)
model.cuda()

# 한번 실행해보기
#text, attention_mask, y = train_dataset[0]
#model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [14]:
try:
  model.load_state_dict(torch.load("model.pt"))
except:
  print("error - model.load_state_dict(torch.load('model.pt'))")
else:
  print("success - model.load_state_dict(torch.load('model.pt'))")

error - model.load_state_dict(torch.load('model.pt'))


In [15]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

# 학습(Learn) 하기

In [16]:
# koelectra-small-v3-discriminator
#epochs = 16
#batch_size = 128

# koelectra-base-v3-discriminator
epochs = 4
batch_size = 32

In [17]:
optimizer = AdamW(model.parameters(), 
                  lr=1e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                  )
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [18]:
losses = []
accuracies = []

# 그래디언트 초기화
# model.zero_grad()

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  # 훈련모드로 변경
  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward() # Backward 수행으로 그래디언트 계산
    #xm.optimizer_step(optimizer, barrier=True)  # TPU 사용시 코드
    optimizer.step() # 그래디언트를 통해 가중치 파라미터 업데이트
    #model.zero_grad() # 그래디언트 초기화

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 61.51964756846428 Accuracy: tensor(0.6750, device='cuda:0')
Batch Loss: 102.30244560539722 Accuracy: tensor(0.7552, device='cuda:0')
Batch Loss: 140.21540582180023 Accuracy: tensor(0.7807, device='cuda:0')
Batch Loss: 174.5535209774971 Accuracy: tensor(0.7972, device='cuda:0')
Batch Loss: 206.1613669320941 Accuracy: tensor(0.8109, device='cuda:0')
Batch Loss: 237.5843553096056 Accuracy: tensor(0.8206, device='cuda:0')
Batch Loss: 269.36846389621496 Accuracy: tensor(0.8270, device='cuda:0')
Batch Loss: 297.55991461127996 Accuracy: tensor(0.8348, device='cuda:0')
Batch Loss: 323.8522268459201 Accuracy: tensor(0.8411, device='cuda:0')
Batch Loss: 352.76889602839947 Accuracy: tensor(0.8452, device='cuda:0')
Batch Loss: 382.39337415993214 Accuracy: tensor(0.8481, device='cuda:0')
Batch Loss: 418.0976305305958 Accuracy: tensor(0.8457, device='cuda:0')
Batch Loss: 448.6676142588258 Accuracy: tensor(0.8475, device='cuda:0')
Batch Loss: 475.4164782240987 Accuracy: tensor(0.8502, dev

Batch Loss: 20.469059325754642 Accuracy: tensor(0.9122, device='cuda:0')
Batch Loss: 43.10673476755619 Accuracy: tensor(0.9100, device='cuda:0')
Batch Loss: 64.61383783072233 Accuracy: tensor(0.9119, device='cuda:0')
Batch Loss: 86.28399500623345 Accuracy: tensor(0.9128, device='cuda:0')
Batch Loss: 108.31215736269951 Accuracy: tensor(0.9114, device='cuda:0')
Batch Loss: 129.3306059166789 Accuracy: tensor(0.9124, device='cuda:0')
Batch Loss: 152.0452896617353 Accuracy: tensor(0.9113, device='cuda:0')
Batch Loss: 169.78902407363057 Accuracy: tensor(0.9138, device='cuda:0')
Batch Loss: 192.54697850346565 Accuracy: tensor(0.9130, device='cuda:0')
Batch Loss: 214.08002047985792 Accuracy: tensor(0.9134, device='cuda:0')
Batch Loss: 234.89532070606947 Accuracy: tensor(0.9135, device='cuda:0')
Batch Loss: 256.2448456734419 Accuracy: tensor(0.9135, device='cuda:0')
Batch Loss: 277.629701025784 Accuracy: tensor(0.9131, device='cuda:0')
Batch Loss: 299.1916187554598 Accuracy: tensor(0.9131, devi

Batch Loss: 17.456662176176906 Accuracy: tensor(0.9312, device='cuda:0')
Batch Loss: 38.09088042937219 Accuracy: tensor(0.9191, device='cuda:0')
Batch Loss: 54.34141509793699 Accuracy: tensor(0.9240, device='cuda:0')
Batch Loss: 69.94334339722991 Accuracy: tensor(0.9271, device='cuda:0')
Batch Loss: 86.77314790338278 Accuracy: tensor(0.9276, device='cuda:0')
Batch Loss: 103.29136085510254 Accuracy: tensor(0.9291, device='cuda:0')
Batch Loss: 121.06823845952749 Accuracy: tensor(0.9287, device='cuda:0')
Batch Loss: 137.82751786522567 Accuracy: tensor(0.9296, device='cuda:0')
Batch Loss: 154.65645895712078 Accuracy: tensor(0.9299, device='cuda:0')
Batch Loss: 172.93823834322393 Accuracy: tensor(0.9297, device='cuda:0')
Batch Loss: 187.98401202820241 Accuracy: tensor(0.9307, device='cuda:0')
Batch Loss: 205.5930521581322 Accuracy: tensor(0.9308, device='cuda:0')
Batch Loss: 221.21808709949255 Accuracy: tensor(0.9312, device='cuda:0')
Batch Loss: 238.67486465722322 Accuracy: tensor(0.9314, 

Batch Loss: 12.052960834465921 Accuracy: tensor(0.9556, device='cuda:0')
Batch Loss: 25.449409219436347 Accuracy: tensor(0.9520, device='cuda:0')
Batch Loss: 39.63708133157343 Accuracy: tensor(0.9504, device='cuda:0')
Batch Loss: 52.52302861493081 Accuracy: tensor(0.9506, device='cuda:0')
Batch Loss: 64.32652057521045 Accuracy: tensor(0.9514, device='cuda:0')
Batch Loss: 74.8014752632007 Accuracy: tensor(0.9532, device='cuda:0')
Batch Loss: 86.15559925418347 Accuracy: tensor(0.9541, device='cuda:0')
Batch Loss: 98.06041422765702 Accuracy: tensor(0.9540, device='cuda:0')
Batch Loss: 111.06386433774605 Accuracy: tensor(0.9530, device='cuda:0')
Batch Loss: 124.06018209690228 Accuracy: tensor(0.9527, device='cuda:0')
Batch Loss: 136.83426664257422 Accuracy: tensor(0.9526, device='cuda:0')
Batch Loss: 149.68711510067806 Accuracy: tensor(0.9525, device='cuda:0')
Batch Loss: 163.65163834718987 Accuracy: tensor(0.9521, device='cuda:0')
Batch Loss: 177.2831005207263 Accuracy: tensor(0.9517, dev

In [26]:
losses, accuracies

([1302.4959189482033, 958.4553683698177, 776.6318214610219, 614.4949332601391],
 [tensor(0.8787, device='cuda:0'),
  tensor(0.9159, device='cuda:0'),
  tensor(0.9325, device='cuda:0'),
  tensor(0.9482, device='cuda:0')])

# 테스트 데이터셋 정확도 확인하기

In [ ]:
# 평가모드로 변경
model.eval()

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# 모델 저장하기

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "model.pt")

# 과제용 데이터 예측 및 맵핑

In [ ]:
#과제용 데이터 예측
# 데이터 로딩
batchSize = 16
sample_loader = DataLoader(sample_dataset, batch_size=batchSize, shuffle=False)

sample_result = sample_dataset.dataset.copy(deep=True)
print(sample_result)

#평가모드로 변경
model.eval()

idx_s = 0
idx_e = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(sample_loader):
  #y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)

  rsList = list(map(int, predicted)) # 결과를 한번에 저장하기 위해 LIST로 변환 처리
  global idx_s, idx_e
  idx_e += len(rsList) #해당 배치구간내에 index 끝값 계산
  #print("index==", idx_s, idx_e)
  #print("--start-- sample_result['predict_label'][idx_s : idx_e]==\n" , sample_result['predict_label'][idx_s : idx_e] )
  sample_result['Predicted'][idx_s : idx_e] = rsList  #배치구간을 한번에 업데이트
  #print("--end-- sample_result['predict_label'][idx_s : idx_e]==\n" , sample_result['predict_label'][idx_s : idx_e] )
  idx_s += len(rsList) #해당 배치구간내에 index 시작값은 윗줄 처리 후 증가

  #test_correct += (predicted == y_batch).sum()
  #test_total += len(y_batch)

# print("Accuracy:", test_correct.float() / test_total)
print(sample_result)


In [23]:
#torch.cuda.empty_cache() #GPU 캐쉬 데이터 삭제

# 결과 파일 저장

In [ ]:
# 주어진 데이터의 결과를 파일로 저장
#sample_csv = sample_result.to_csv('sample.csv')
sample_csv = sample_result.to_csv('sample.csv',sep=',',na_rep='NaN', columns=['Id','Predicted'],index=False)

In [ ]:
# 파일을 PC로 다운로드 하기
from google.colab import files
files.download('sample.csv')
